In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import pdb
import numpy as np
import numpy.random as npr
import tensorflow as tf
from tensorflow.python.ops import parallel_for as pfor
import sys
import json
import re
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from scipy import stats
from numpy import linalg as LA
from sklearn import linear_model
from sklearn.manifold import MDS
from sklearn.neighbors import DistanceMetric
from scipy.spatial import distance
from sklearn.cluster import KMeans


import getpass
ui = getpass.getuser()
if ui == 'laura':
    p = '/home/laura'
elif ui == 'lauradriscoll':
    p = '/Users/lauradriscoll/Documents'

net = 'stepnet'
PATH_YANGNET = os.path.join(p,'code/multitask-nets',net) 

sys.path.insert(0, PATH_YANGNET)
from task import generate_trials, rule_name, rule_index_map, rules_dict
from network import Model, get_perf, FixedPoint_Model
import tools
from tools_lnd import get_T_inds, plot_FP_jitter, plot_N, TDR_axes, out_axes
from tools_lnd import make_axes, same_stim_trial, generate_Beta_epoch, take_names
from tools_lnd import make_h_trial_rule, get_model_params, gen_trials_from_model_dir, gen_X_from_model_dir

In [2]:
model_n = 0
task_list = ['fdgo','fdanti','delaygo','delayanti']
rule = task_list[0]
which_net = 'l2w0001'
fldr = '4_tasks'
supp = []#'long_train'

if fldr == '4_tasks':
    s = '_'
    rule_trains_str = s.join(task_list)
    file_spec = os.path.join(fldr,which_net+'_'+rule_trains_str)
else:
    file_spec = which_net
    
dir_specific_all = os.path.join('crystals','softplus',file_spec)#,supp)
    
m_all = os.path.join(p,'data/rnn/multitask/',net,dir_specific_all,str(model_n))
h_all,trial_all,tasks_all = make_h_trial_rule(m_all,mode = 'test',noise_on = False)

figpath = os.path.join(p,'code','overleaf','task_schematic_multitask','figs','interp_fps',file_spec,str(model_n))
if not os.path.exists(figpath):
    os.makedirs(figpath)

Overwrite sigma_rec with 0.000
0.0
INFO:tensorflow:Restoring parameters from /Users/lauradriscoll/Documents/data/rnn/multitask/stepnet/crystals/softplus/4_tasks/l2w0001_fdgo_fdanti_delaygo_delayanti/0/model.ckpt
Model restored from file: /Users/lauradriscoll/Documents/data/rnn/multitask/stepnet/crystals/softplus/4_tasks/l2w0001_fdgo_fdanti_delaygo_delayanti/0/model.ckpt


In [65]:
########## X EPOCH ###########

fig = plt.figure(figsize=(6,6),tight_layout=True,facecolor='white')
t_set = [0,0]
trial_set = range(0,80,10)
rule_set = ['delaygo',]
epoch_set = ['stim1','delay1']
step_file = 'interp_epochs'
linestyle = ['-','--']
axs ='tdr'

ri = 0
rule = rule_set[ri]
epoch_axs = 0

f = os.path.join(m,'tf_fixed_pts_all_init',rule,epoch_set[epoch_axs]+'_'+str(t_set[epoch_axs])+'.0.npz')
# D_use,_,_ = make_FP_axs(f,m_all,rule,epoch_set[epoch_axs],axs =axs, clust = 'False')

use_yinds = [0,1]
axes_label1 = Y_labels[use_yinds[0]]
axes_label2 = Y_labels[use_yinds[1]]
D_use = Beta_qr[:,use_yinds]

cmap_grad = plt.get_cmap('plasma')
cmap_discrete = ['darkorchid','orange']


trial = gen_trials_from_model_dir(m_all,rule_set[0],noise_on = False)

for ei in range(len(epoch_set)):
    ax = plt.subplot(111)
    
    epoch = epoch_set[ei]
    f = os.path.join(m,'tf_fixed_pts_all_init',rule,epoch+'_'+str(t_set[ri])+'.0.npz')
    fp_struct = np.load(f)
    sorted_fps = fp_struct['xstar']
    qvals = fp_struct['qstar']
    fp_project = np.dot(sorted_fps,D_use)
#     print(np.log10(qvals))
    
    stable_fp = np.log10(qvals)<-6
    plt.plot(fp_project[stable_fp==1,0],fp_project[stable_fp==1,1],'o',c = cmap_discrete[ei],
             markersize = 10, alpha = 1, linewidth = 2)
    plt.plot(fp_project[stable_fp==0,0],fp_project[stable_fp==0,1],'o',c = cmap_discrete[ei],
             markersize = 10, markerfacecolor = 'none', alpha = 1, linewidth = 2)
    
    _,x = gen_X_from_model_dir(m,trial)
    T_inds = get_T_inds(trial,epoch)
    x_epoch = np.transpose(x[:,:,T_inds],(1,2,0))
    plot_N(x_epoch[trial_set,:,:],D_use.T,trial.y_loc[-1,trial_set],linewidth = 2, linestyle = linestyle[ei])

    for step_i in range(20):
        f = rule_set[0]+'_'+epoch_set[0]+'_'+epoch_set[1]+'_'+str(t_set[ri])+'_step_'+str(step_i)
        filename = os.path.join(m,step_file,rule_set[0],f+'.npz')
        fp_struct = np.load(filename)
        step_fps = fp_struct['xstar']
#         print(len(step_fps))

        fp_project = np.dot(step_fps,D_use)
        plt.plot(fp_project[:,0],fp_project[:,1],'.',markersize = 5,c = cmap_grad(step_i/n_interp))
        
tit = 'X EPOCH' + ' Stim vs. Memory'
plt.title(tit,fontsize = 18)
plt.xlabel(axs+' 1',fontsize = 18)
plt.ylabel(axs+' 2',fontsize = 18)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

if not os.path.exists(os.path.join(figpath,step_file)):
    os.makedirs(os.path.join(figpath,step_file))
figname = axes_label1+'_'+axes_label2+'_'+rule+'_'+epoch+'_'+str(t_set[0])+'_'+tit
# figname = axs+'_'+rule+'_'+epoch+'_'+str(t_set[0])+'_'+tit
plt.savefig(os.path.join(figpath,step_file,figname+'.pdf'))
plt.savefig(os.path.join(figpath,step_file,figname+'.png'))
plt.show()

Overwrite sigma_rec with 0.000
0.0
INFO:tensorflow:Restoring parameters from /Users/lauradriscoll/Documents/data/rnn/multitask/stepnet/crystals/softplus/4_tasks/l2w0001_fdgo_fdanti_delaygo_delayanti/0/model.ckpt
Model restored from file: /Users/lauradriscoll/Documents/data/rnn/multitask/stepnet/crystals/softplus/4_tasks/l2w0001_fdgo_fdanti_delaygo_delayanti/0/model.ckpt


In [ ]:
########## X TASK ###########
# from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits import mplot3d
fig = plt.figure(figsize=(10,10),tight_layout=True,facecolor='white')
t_set = [0,180]
trial_num = [0,40]
trial_set = range(0,80,10)
rule_set = ['delaygo','delayanti']
epoch_set = ['stim1',]
step_file = 'interp_tasks'
linestyle = ['-','--']
axs = 'pca'

trial_str = '_'.join(str(t) for t in t_set)
rule_str = '_'.join(str(r) for r in rule_set)

ei = 0
epoch = epoch_set[ei]

rule_axs = 0
# f = os.path.join(m,'tf_fixed_pts_all_init',rule_set[rule_axs],epoch+'_'+str(t_set[rule_axs])+'.0.npz')
# D_use,_,_ = make_FP_axs(f,m_all,rule_set[rule_axs],epoch,axs =axs, clust = 'False')
# axes_label1 = 'pca_1'
# axes_label2 = 'pca_2'
# axes_label3 = 'pca_3'

use_yinds = [0,2,3]
axes_label1 = Y_labels[use_yinds[0]]
axes_label2 = Y_labels[use_yinds[1]]
axes_label3 = Y_labels[use_yinds[2]]
D_use = Beta_qr[:,use_yinds]

cmap_grad = plt.get_cmap('plasma')
cmap_discrete = ['darkorchid','orange']
ax = fig.add_subplot(111, projection='3d')
w_color = .005
ax.w_xaxis.set_pane_color ((w_color, w_color, w_color, w_color))
ax.w_yaxis.set_pane_color ((w_color, w_color, w_color, w_color))
ax.w_zaxis.set_pane_color ((w_color, w_color, w_color, w_color))



for ri in range(len(rule_set)):
    rule = rule_set[ri]
    trial = gen_trials_from_model_dir(m_all,rule,noise_on = False)
#     ax = plt.subplot(111)
    
    f = os.path.join(m,'tf_fixed_pts_all_init',rule,epoch+'_'+str(t_set[ri])+'.0.npz')
    fp_struct = np.load(f)
    sorted_fps = fp_struct['xstar']
    qvals = fp_struct['qstar']
    fp_project = np.dot(sorted_fps,D_use)
#     print(np.log10(qvals))

    if ri<2:
        plot_FP_jitter_3D(m_all,D_use,rule,trial_num[0],epoch,sorted_fps,range(len(sorted_fps)),qvals,
               rand_step_coef = rand_step_coef, al = al, linestyle = linestyle, n_steps = 100, 
                   n_jit = 5,unstable_qlim = -10,c = cmap_discrete[ri])
    
    stable_fp = np.log10(qvals)<-10
    ax.plot3D(fp_project[stable_fp==1,0],fp_project[stable_fp==1,1],fp_project[stable_fp==1,2],
             'o',c = cmap_discrete[ri],markersize = 10, alpha = 1,linewidth = 2)
    ax.plot3D(fp_project[stable_fp==0,0],fp_project[stable_fp==0,1],fp_project[stable_fp==0,2],
             'o',c = cmap_discrete[ri],markersize = 10, markerfacecolor = 'none', alpha = 1,linewidth = 2)
    
    _,x = gen_X_from_model_dir(m,trial)
    T_inds = get_T_inds(trial,epoch)
    x_epoch = np.transpose(x[:,:,T_inds],(1,2,0))
    plot_N3D(x_epoch[trial_set,:,:],D_use.T,trial.y_loc[-1,trial_set],linewidth = 4, linestyle = linestyle[ri])

    for step_i in range(20):
        f = rule_set[0]+'_'+rule_set[1]+'_'+epoch_set[0]+'_'+str(t_set[0])+'_step_'+str(step_i)
        filename = os.path.join(m,step_file,rule_set[0]+'_'+rule_set[1],f+'.npz')
        fp_struct = np.load(filename)
        step_fps = fp_struct['xstar']
#         print(len(step_fps))

        fp_project = np.dot(step_fps,D_use)
        ax.plot3D(fp_project[:,0],fp_project[:,1],fp_project[:,2],'.',markersize = 5,c = cmap_grad(step_i/n_interp))
#         plt.show()

tit = 'X TASK ' + epoch_set[ei]+' '+rule_set[0]+' vs.'+rule_set[1]
# plt3D.title(tit,fontsize = 18)
ax.set_xlabel(axes_label1,fontsize = 18)
ax.set_ylabel(axes_label2,fontsize = 18)
ax.set_zlabel(axes_label3,fontsize = 18)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# if not os.path.exists(os.path.join(figpath,step_file)):
#     os.makedirs(os.path.join(figpath,step_file))
    
# figname = axs+'_'+rule+'_'+epoch+'_'+str(t_set[0])+'_'+tit
# figname = axes_label1+'_'+axes_label2+'_'+rule+'_'+epoch+'_'+str(t_set[0])+'_'+tit
# plt.savefig(os.path.join(figpath,step_file,figname+'.pdf'))
# plt.savefig(os.path.join(figpath,step_file,figname+'.png'))
# plt.show()

In [ ]:
def plot_N3D(X, D, clist, linewidth = 1, alpha = .5, linestyle = '-'):
    """Plot activity is some 2D space.

        Args:
            X: neural activity in Trials x Time x Neurons
            D: Neurons x 2 plotting dims
        """

    cmap=plt.get_cmap('rainbow')
    S = np.shape(X)[0]
    
    for s in range(S):

        if len(clist)==1:
            c = clist
        else:
            c = cmap(int(clist[s])/max(clist))

        X_trial = np.dot(X[s,:,:],D.T)
        ax.scatter3D(X_trial[-1,0],X_trial[-1,1],X_trial[-1,2],'^',c = c, linewidth = linewidth, alpha = .5)
        ax.plot3D(X_trial[:,0],X_trial[:,1],X_trial[:,2],linestyle,c = c, linewidth = linewidth, alpha = .5)
        ax.scatter3D(X_trial[0,0],X_trial[0,1],X_trial[0,2],'.',c = c, linewidth = linewidth, alpha = .5)

In [ ]:
def plot_FP_jitter_3D(m,D_use,rule,t_num,fp_epoch,sorted_fps,fp_inds,qvals,
                   xlabel = 'FP set PC1',ylabel = 'FP set PC2',rand_step_coef = 0.1,n_steps = 100,
                   lw = 3,al = .6,linestyle = '-',n_jit = 0,unstable_qlim = -6,c = 'k'):

    cmap=plt.get_cmap('rainbow')
        
    model = Model(m)
    with tf.Session() as sess:
        model.restore()
        model._sigma=0
        hparams = model.hp
        alpha = hparams['dt']/hparams['tau']
        var_list = model.var_list
        params = [sess.run(var) for var in var_list]

        trial = generate_trials(rule, hparams, mode='test',noise_on=False)
        feed_dict = tools.gen_feed_dict(model, trial, hparams)
        h_tf, _ = sess.run([model.h, model.y_hat], feed_dict=feed_dict) #(n_time, n_condition, n_neuron)
        T,S,N = np.shape(h_tf)
        T_inds = get_T_inds(trial,fp_epoch) # grab epoch time indexing
        
        for fp_ind in fp_inds:

            x_t = np.matlib.repmat(trial.x[T_inds[1],t_num,:],n_steps,1)
            for jit in range(n_jit):
                h0 = sorted_fps[fp_ind,:] + rand_step_coef*npr.randn(N)
                h_t = vanilla_run_with_h0(params, x_t, h0, hparams)
                jitter = np.dot(h_t,D_use)
                ax.plot3D(jitter[:,0],jitter[:,1],jitter[:,2],'-',c = c,linewidth = .1)
                
        
        for jit in range(n_jit):
            h0 = h_tf[T_inds[0],t_num,:] + rand_step_coef*npr.randn(N)
            h_t = vanilla_run_with_h0(params, x_t, h0, hparams)
            jitter = np.dot(h_t,D_use)
            ax.plot3D(jitter[:,0],jitter[:,1],jitter[:,2],'-',c = c,linewidth = .1)


In [ ]:
def plot_N(X, D, clist, linewidth = 1, alpha = .5, linestyle = '-'):
    """Plot activity is some 2D space.

        Args:
            X: neural activity in Trials x Time x Neurons
            D: Neurons x 2 plotting dims
        """

    cmap=plt.get_cmap('hsv')
    S = np.shape(X)[0]
    
    for s in range(S):

        if len(clist)==1:
            c = clist
        else:
            c = cmap(int(clist[s])/max(clist))

        X_trial = np.dot(X[s,:,:],D.T)
        plt.plot(X_trial[-1,0],X_trial[-1,1],'^',c = c, linewidth = linewidth, alpha = alpha)
        plt.plot(X_trial[:,0],X_trial[:,1],linestyle,c = c, linewidth = linewidth, alpha = alpha)
        plt.plot(X_trial[0,0],X_trial[0,1],'.',c = c, linewidth = linewidth, alpha = alpha)

In [ ]:
########## X STIM ###########

fig = plt.figure(figsize=(6,6),tight_layout=True,facecolor='white')
t_set = [0,36]
trial_set = range(0,80,10)
rule_set = ['delaygo',]
epoch_set = ['stim1',]
step_file = 'interp_stim'
axs = 'tdr'

ei = 0
epoch = epoch_set[ei]
ri = 0
rule = rule_set[ei]
trial = gen_trials_from_model_dir(m_all,rule,noise_on = False)

stim_axs = 0
f = os.path.join(m,'tf_fixed_pts_all_init',rule,epoch+'_'+str(t_set[stim_axs])+'.0.npz')
D_use,_,_ = make_FP_axs(f,m_all,rule,epoch,axs =axs, clust = 'False')

cmap_grad = plt.get_cmap('plasma')
cmap_discrete = ['darkorchid','orange']

plot_N(x_epoch[trial_set,:,:],D_use.T,trial.y_loc[-1,trial_set],linewidth = 10, 
       linestyle = linestyle[ri], alpha = .2)
    
for si in range(len(t_set)):
    ax = plt.subplot(111)
    
    f = os.path.join(m,'tf_fixed_pts_all_init',rule,epoch+'_'+str(t_set[si])+'.0.npz')
    fp_struct = np.load(f)
    sorted_fps = fp_struct['xstar']
    qvals = fp_struct['qstar']
    fp_project = np.dot(sorted_fps,D_use)
#     print(np.log10(qvals))
    
    stable_fp = np.log10(qvals)<-6
    plt.plot(fp_project[stable_fp==1,0],fp_project[stable_fp==1,1],'o',c = cmap_discrete[si],
             markersize = 10, alpha = 1,linewidth = 5)
    plt.plot(fp_project[stable_fp==0,0],fp_project[stable_fp==0,1],'o',c = cmap_discrete[si],
             markersize = 10, markerfacecolor = 'none', alpha = 1,linewidth = 5)
    
    _,x = gen_X_from_model_dir(m,trial)
    T_inds = get_T_inds(trial,epoch)
    x_epoch = np.transpose(x[:,:,T_inds],(1,2,0))

    X_trial = np.dot(x_epoch[trial_set[si],:,:],D_use)
    plt.plot(X_trial[:,0],X_trial[:,1],c = cmap_discrete[si],linewidth = 2, 
           linestyle = linestyle[ri], alpha = 1)

    for step_i in range(20):
        
        f = rule+'_'+epoch+'_'+str(t_set[0])+'_'+str(t_set[1])+'_step_'+str(step_i)
        filename = os.path.join(m,step_file,rule,f+'.npz')
        fp_struct = np.load(filename)
        step_fps = fp_struct['xstar']

        fp_project = np.dot(step_fps,D_use)
        plt.plot(fp_project[:,0],fp_project[:,1],'.',markersize = 5,c = cmap_grad(step_i/n_interp))
        
tit = 'X STIM'
plt.title(tit,fontsize = 18)
plt.xlabel(axs+' 1',fontsize = 18)
plt.ylabel(axs+' 2',fontsize = 18)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

if not os.path.exists(os.path.join(figpath,step_file)):
    os.makedirs(os.path.join(figpath,step_file))
figname = axs+'_'+rule+'_'+epoch+'_'+str(t_set[0])+'_'+str(t_set[1])+'_'+tit+'.pdf'
plt.savefig(os.path.join(figpath,step_file,figname))
figname = axs+'_'+rule+'_'+epoch+'_'+str(t_set[0])+'_'+str(t_set[1])+'_'+tit+'.png'
plt.savefig(os.path.join(figpath,step_file,figname))
plt.show()

In [ ]:
########## X TASK ###########

fig = plt.figure(figsize=(6,6),tight_layout=True,facecolor='white')
t_set = [0,0]
trial_set = range(0,80,10)
rule_set = ['fdgo','delaygo']
epoch_set = ['stim1',]
step_file = 'interp_tasks'
linestyle = ['-','--']
axs = 'pca'

trial_str = '_'.join(str(t) for t in t_set)
rule_str = '_'.join(str(r) for r in rule_set)

ei = 0
epoch = epoch_set[ei]

rule_axs = 0
# f = os.path.join(m,'tf_fixed_pts_all_init',rule_set[rule_axs],epoch+'_'+str(t_set[rule_axs])+'.0.npz')
# D_use,_,_ = make_FP_axs(f,m_all,rule_set[rule_axs],epoch,axs =axs, clust = 'False')
use_yinds = [0,3]
axes_label1 = Y_labels[use_yinds[0]]
axes_label2 = Y_labels[use_yinds[1]]
D_use = Beta_qr[:,use_yinds]

cmap_grad = plt.get_cmap('plasma')
cmap_discrete = ['darkorchid','orange']

for ri in range(len(rule_set)):
    rule = rule_set[ri]
    trial = gen_trials_from_model_dir(m_all,rule,noise_on = False)
    ax = plt.subplot(111)
    
    f = os.path.join(m,'tf_fixed_pts_all_init',rule,epoch+'_'+str(t_set[ri])+'.0.npz')
    fp_struct = np.load(f)
    sorted_fps = fp_struct['xstar']
    qvals = fp_struct['qstar']
    fp_project = np.dot(sorted_fps,D_use)
#     print(np.log10(qvals))
    
    stable_fp = np.log10(qvals)<-6
    plt.plot(fp_project[stable_fp==1,0],fp_project[stable_fp==1,1],'o',c = cmap_discrete[ri],
             markersize = 10, alpha = 1,linewidth = 2)
    plt.plot(fp_project[stable_fp==0,0],fp_project[stable_fp==0,1],'o',c = cmap_discrete[ri],
             markersize = 10, markerfacecolor = 'none', alpha = 1,linewidth = 2)
    
    _,x = gen_X_from_model_dir(m,trial)
    T_inds = get_T_inds(trial,epoch)
    x_epoch = np.transpose(x[:,:,T_inds],(1,2,0))
    plot_N(x_epoch[trial_set,:,:],D_use.T,trial.y_loc[-1,trial_set],linewidth = 2, linestyle = linestyle[ri])

    for step_i in range(20):
        f = rule_set[0]+'_'+rule_set[1]+'_'+epoch_set[0]+'_'+str(t_set[0])+'_step_'+str(step_i)
        filename = os.path.join(m,step_file,rule_set[0]+'_'+rule_set[1],f+'.npz')
        fp_struct = np.load(filename)
        step_fps = fp_struct['xstar']
#         print(len(step_fps))

        fp_project = np.dot(step_fps,D_use)
        plt.plot(fp_project[:,0],fp_project[:,1],'.',markersize = 5,c = cmap_grad(step_i/n_interp))
#         plt.show()

tit = 'X TASK ' + epoch_set[ei]+' '+rule_set[0]+' vs.'+rule_set[1]
plt.title(tit,fontsize = 18)
plt.xlabel(axes_label1,fontsize = 18)
plt.ylabel(axes_label2,fontsize = 18)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

if not os.path.exists(os.path.join(figpath,step_file)):
    os.makedirs(os.path.join(figpath,step_file))
    
figname = axs+'_'+rule+'_'+epoch+'_'+str(t_set[0])+'_'+tit
figname = axes_label1+'_'+axes_label2+'_'+rule+'_'+epoch+'_'+str(t_set[0])+'_'+tit
plt.savefig(os.path.join(figpath,step_file,figname+'.pdf'))
plt.savefig(os.path.join(figpath,step_file,figname+'.png'))
plt.show()

In [ ]:
########## X TASK ###########

fig = plt.figure(figsize=(6,6),tight_layout=True,facecolor='white')
t_set = [0,180]
trial_set = range(0,80,10)
rule_set = ['fdanti','delayanti']
epoch_set = ['stim1',]
step_file = 'interp_tasks'
linestyle = ['-','--']
axs = 'pca'

trial_str = '_'.join(str(t) for t in t_set)
rule_str = '_'.join(str(r) for r in rule_set)

ei = 0
epoch = epoch_set[ei]

rule_axs = 0
# f = os.path.join(m,'tf_fixed_pts_all_init',rule_set[rule_axs],epoch+'_'+str(t_set[rule_axs])+'.0.npz')
# D_use,_,_ = make_FP_axs(f,m_all,rule_set[rule_axs],epoch,axs =axs, clust = 'False')
use_yinds = [0,3]
axes_label1 = Y_labels[use_yinds[0]]
axes_label2 = Y_labels[use_yinds[1]]
D_use = Beta_qr[:,use_yinds]

cmap_grad = plt.get_cmap('plasma')
cmap_discrete = ['darkorchid','orange']

for ri in range(len(rule_set)):
    rule = rule_set[ri]
    trial = gen_trials_from_model_dir(m_all,rule,noise_on = False)
    ax = plt.subplot(111)
#     fig = plt.figure(figsize=(16,8),tight_layout=True,facecolor='white')
    
    f = os.path.join(m,'tf_fixed_pts_all_init',rule,epoch+'_'+str(t_set[ri])+'.0.npz')
    fp_struct = np.load(f)
    sorted_fps = fp_struct['xstar']
    qvals = fp_struct['qstar']
    fp_project = np.dot(sorted_fps,D_use)
#     print(np.log10(qvals))
    
    stable_fp = np.log10(qvals)<-6
    plt.plot(fp_project[stable_fp==1,0],fp_project[stable_fp==1,1],'o',c = cmap_discrete[ri],
             markersize = 10, alpha = 1,linewidth = 2)
    plt.plot(fp_project[stable_fp==0,0],fp_project[stable_fp==0,1],'o',c = cmap_discrete[ri],
             markersize = 10, markerfacecolor = 'none', alpha = 1,linewidth = 2)
    
    _,x = gen_X_from_model_dir(m,trial)
    T_inds = get_T_inds(trial,epoch)
    x_epoch = np.transpose(x[:,:,T_inds],(1,2,0))
    plot_N(x_epoch[trial_set,:,:],D_use.T,trial.y_loc[-1,trial_set],linewidth = 2, linestyle = linestyle[ri])

    for step_i in range(20):
        f = rule_str+'_'+epoch_set[0]+'_'+trial_str+'_step_'+str(step_i)
        filename = os.path.join(m,step_file,rule_str,f+'.npz')
        fp_struct = np.load(filename)
        step_fps = fp_struct['xstar']
#         print(len(step_fps))

        fp_project = np.dot(step_fps,D_use)
        plt.plot(fp_project[:,0],fp_project[:,1],'.',markersize = 5,c = cmap_grad(step_i/n_interp))
#         plt.show()

tit = 'X TASK ' + epoch_set[ei] + ' ' + rule_set[0] + ' vs. ' + rule_set[1]
plt.title(tit,fontsize = 18)
plt.xlabel(axs+' 1',fontsize = 18)
plt.ylabel(axs+' 2',fontsize = 18)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

if not os.path.exists(os.path.join(figpath,step_file)):
    os.makedirs(os.path.join(figpath,step_file))
    
figname = axes_label1+'_'+axes_label2+'_'+rule_str+'_'+epoch+t_str
plt.savefig(os.path.join(figpath,step_file,figname+'.pdf'))
plt.savefig(os.path.join(figpath,step_file,figname+'.png'))
plt.show()